[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a template for your DIY Covid Dashboard, to which you can add the code you developed in the previous notebooks. The dashboard will be displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. Contrary to the other libraries we have seen, the ```voila``` package must be installed using *pip* or *conda* but it does not need to be imported - it rather acts at the level of the notebook server. Package ```voila``` is already installed on the QMUL JupyterHub as well as in the Binder - to install it locally, follow the [instructions](https://voila.readthedocs.io/en/stable/install.html) online.

Broadly speaking, Voila acts by **running all the cells in your notebook** when the dashboard is first loaded; it then hides all code cells and displays all markdown cells and any outputs, including widgets. However, the code is still there in the background and handles any interaction with the widgets. To view this dashboard template rendered in Voila click [here](https://mybinder.org/v2/gh/fsmeraldi/diy-covid19dash/main?urlpath=%2Fvoila%2Frender%2FDashboard.ipynb).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

## Load initial data from disk

You should include "canned" data in ```.json``` files along with your dashboard. When the dashboard starts, it should load that data (the code below will be hidden when the dashboard is rendered by Voila).

In [20]:
# Load JSON files and store the raw data in some variable. Edit as appropriate
jsondata= {}
with open("vaccines_and_deaths.json", "r") as INFILE:
    data=json.load(INFILE)

## Wrangle the data

The dashboard should contain the logic to wrangle the raw data into a ```DataFrame``` (or more than one, as required) that will be used for plotting. The wrangling code should be put into a function and called on the data from the JSON file (we'll need to call it again on any data downloaded from the API).  In this template, we just pretend we are wrangling ```rawdata``` and generate a dataframe with some random data

In [26]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")
    

def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
   
    datalist=data['data']
    
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
  
 
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
 
    index=pd.date_range(startdate, enddate, freq='D')
    vaccines_and_deathsdf=pd.DataFrame(index=index, columns=['new vaccines', 'hospital', 'cumulative deaths'])
   
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['new vaccines', 'hospital', 'cumulative deaths']:
            # check that nothing is there yet - just in case some dates are duplicated,
            # maybe with data for different columns in each entry
            if pd.isna(vaccines_and_deathsdf.loc[date, column]): 
                # replace None with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                vaccines_and_deathsdf.loc[date, column]=value
                
    # fill in any remaining "holes" due to missing dates
    vaccines_and_deathsdf.fillna(0.0, inplace=True)
                
    return vaccines_and_deathsdf


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(data) # df is the dataframe for plotting

## Download current data

Give your users an option to refresh the dataset - a "refresh" button will do. The button callback should
* call the code that accesses the API and download some fresh raw data;
* wrangle that data into a dataframe and update the corresponding (global) variable for plotting;
* optionally: force a redraw of the graph and give the user some fredback.

Once you get it to work, you may want to wrap your API call inside an exception handler, so that the user is informed, the "canned" data are not overwritten and nothing crashes if for any reason the server cannot be reached or data are not available.

After you refresh the data, graphs will not update until the user interacts with a widget. You can trick ```iPywidgets``` into redrawing the graph by simulating interaction, as in the ```refresh_graph``` function we define in the Graph and Analysis section below.

Clicking on the button below just generates some more random data and refreshes the graph. The button should read *Fetch Data*. If you see anything else, take a deep breath :)

In [51]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api(button):
    
    filters = [
        'areaType=overview', # note each metric-value pair is inside one string
    ]

    # values here are the names of the PHE metrics
    structure = {
        "date": "date",
        "new vaccines": "newVaccinesGivenByPublishDate",
        "hospital": "newAdmissionsRollingRate",
        "cumulative deaths": "newDailyNsoDeathsByDeathDate" 
        }
    
    api = Cov19API(filters=filters, structure=structure)
    
    vaccines_and_deaths=api.get_json()
    apibutton.icon="check"
    apibutton.disabled=True



    return {} # return data read from the AP

In [50]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
# the graph won't refresh until the user interacts with the widget.
# this function simulates the interaction, see Graph and Analysis below.
# you can omit this step in the first instance
refresh_graph()
# after all is done, you can switch the icon on the button to a "check" sign
# and optionally disable the button - it won't be needed again. You can use icons
# "unlink" or "times" and change the button text to "Unavailable" in case the 
# api call fails.
apibutton.icon="check"
# apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Refresh data',
    disabled=False,
    button_style='',
    tooltip='Click to download current Public Health England data',
    icon='download' 
    )


# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)

# run all cells before clicking on this button

Button(description='Refresh data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pu…

## Graphs and Analysis

Include at least one graph with interactive controls, as well as some instructions for the user and/or comments on what the graph represents and how it should be explored (this example shows two random walks)

In [58]:
vaccines_and_deaths_df=pd.read_pickle("vaccines_and_deathsdf.pkl")


vaccinescols=wdg.SelectMultiple(
    options=['new vaccines', 'hospital', 'cumulative deaths'], 
    value=['new vaccines', 'hospital', 'cumulative deaths'], 
    rows=3,
    description='vaccines',
    disabled=False
)

def date_picker():
    wdg.DatePicker(
        description='Pick a Date',
        disabled=False
    )



dates = vaccines_and_deaths_df.index.values

options = [(str(date)[:10], date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = wdg.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '600px'}
)

scale=wdg.RadioButtons(
    options=['linear', 'log'],

    description='Scale:',
    disabled=False
)


controls1=wdg.HBox([vaccinescols, scale, selection_range_slider])

def vaccines_to_deaths_graphs(graphcolumns, gdate, gscale):
    
    ncols=len(graphcolumns)
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
        
    if ncols>0:
        vaccines_and_deaths_df.loc[gdate[0]:gdate[1]].plot(logy=logscale, y=list(graphcolumns)) 
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
   
    ncols=len(graphcolumns)
output=wdg.interactive_output(vaccines_to_deaths_graphs, {'graphcolumns': vaccinescols, 'gdate': selection_range_slider, 'gscale': scale})
display(controls1, output)


def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=vaccinescols.value
    vaccinescols.value=() # forces the redraw
    vaccinescols.value=current # now we can change it back


Output()

## Deploying the dashboard

Once your code is ready and you are satisfied with the appearance of the graphs, replace all the text boxes above with the explanations you would like a dashboard user to see. The next step is deploying the dashboard online - there are several [options](https://voila.readthedocs.io/en/stable/deploy.html) for this, we suggest deploying as a [Binder](https://mybinder.org/). This is basically the same technique that has been used to package this tutorial and to deploy this template dashboard. The instructions may seem a bit involved, but the actual steps are surprisingly easy - we will be going through them together during a live session. You will need an account on [GitHub](https://github.com/) for this - if you don't have one already, now it's the time to create it. 

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).